In [1]:
%load_ext autoreload
%autoreload 2

In [42]:
import os
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
from spatial_access.Models import AccessCount, AccessModel, AccessSum, AccessTime

In [43]:
pd.options.display.max_rows = 100

In [66]:
import utils
import spatial_access_prep

In [67]:
STATES  = ['AL',
          #'AK',
           'AZ','AR','CA','CO','CT','DE','DC','FL','GA',
          #'HI',
           'ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN',
           'MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH',
           'OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA',
           'WV','WI','WY']

In [138]:
# resources_path = './data/us-wide-moudsCleaned.gpkg'
# us = gpd.read_file(resources_path)
# us['category'].unique()


resources_path = './data/SUT_locations.gpkg'
us = gpd.read_file(resources_path)
us['category'] = 'STU'
us.to_file(resources_path, driver='GPKG')
us['category'].unique()

array(['STU'], dtype=object)

In [89]:
national_shp   = './shapefiles/tract/US/tl_2010_us_ttract10.shp'
outpath = './'
resource = 'STU'

In [90]:
source_col_names = {'idx':'GEOID',
                    'population': 'skip',
                    'lon': 'oX',
                    'lat': 'oY'}
dest_col_names   = {'idx':'ID',
                    'category': 'category',
                    'capacity': 'skip',
                    'lon': 'dX',
                    'lat': 'dY'}

In [92]:
for state in STATES:
#     utils.get_resource(state, 'tract', './')
    access_metrics_file = f'{state}/{state}-{resource[:3]}-access-metrics.csv'
    access_metrics_path = f'./outputs/tract/{access_metrics_file}'
    
    origs_path = f'./inputs/tract/{state}/{state}-origins.csv'
    dests_path = f'./inputs/tract/{state}/{state}-{resource[:3]}-moud-dests.csv'
    matrix_csv = f'./inputs/tract/{state}/{state}-transformed-matrix.csv'
    matrix_tmx = f'./inputs/tract/{state}/{state}-transformed-matrix.tmx'
    
    if not os.path.exists(access_metrics_path):
        
        if not os.path.exists(origs_path):
            spatial_access_prep.create_origins_file(state, outpath, save=True)
            
        spatial_access_prep.create_destinations_file(state, resources_path, './', resource = resource, save=True)
        spatial_access_prep.create_transformed_matrix(state, outpath,resource = resource, save=True)

        accessM = AccessModel(network_type = 'drive',
                         transit_matrix_filename = matrix_csv,
                         sources_filename = origs_path,
                         destinations_filename = dests_path,
                         source_column_names = source_col_names,
                         dest_column_names = dest_col_names)  
        
        if not os.path.exists(matrix_tmx):
            accessM.transit_matrix.write_tmx(matrix_tmx)

        accessT = AccessTime(network_type = 'drive',
                         transit_matrix_filename = matrix_tmx,
                         sources_filename = origs_path,
                         destinations_filename = dests_path,
                         source_column_names = source_col_names,
                         dest_column_names = dest_col_names)  

        accessC = AccessCount(network_type = 'drive',
                         transit_matrix_filename = matrix_tmx,
                         sources_filename = origs_path,
                         destinations_filename = dests_path,
                         source_column_names = source_col_names,
                         dest_column_names = dest_col_names)  

        weights = {resource:[1,1,1]}

        accessT.calculate()
        ttn = accessT.model_results.reset_index()

        accessC.calculate(upper_threshold = 30)
        nCount = accessC.model_results.reset_index()


        accessM.calculate(upper_threshold = 30, category_weight_dict = weights)
        score = accessM.model_results.reset_index()

        accessBup = ttn.merge(nCount)
        accessBup = accessBup.merge(score)

        accessBup.drop(['time_to_nearest_all_categories','count_in_range_all_categories','all_categories_score'], axis=1, inplace=True)
        accessBup = accessBup.rename(columns= {'index':'tract'})
#         accessBup

        accessBup.to_csv(access_metrics_path, index = False)
        print(f'{state} access metrics for {resource} complete!')


Skipping tl_2010_us_state10.zip, it's already downloaded.
Skipping tl_2010_us_state10.zip, it's already extracted.
Skipping tl_2010_13_tract10.zip, it's already downloaded.
Skipping tl_2010_13_tract10.zip, it's already extracted.
Skipping tl_2010_28_tract10.zip, it's already downloaded.
Skipping tl_2010_28_tract10.zip, it's already extracted.
Skipping tl_2010_12_tract10.zip, it's already downloaded.
Skipping tl_2010_12_tract10.zip, it's already extracted.
Skipping tl_2010_01_tract10.zip, it's already downloaded.
Skipping tl_2010_01_tract10.zip, it's already extracted.
Skipping tl_2010_47_tract10.zip, it's already downloaded.
Skipping tl_2010_47_tract10.zip, it's already extracted.
Skipping tl_2010_22_tract10.zip, it's already downloaded.
Skipping tl_2010_22_tract10.zip, it's already extracted.
AL destinations saved to ./inputs/tract/AL/AL-STU-moud-dests.csv


AL transformed matrix saved to ./inputs/tract/AL/AL-transformed-matrix.csv


INFO:spatial_access.BaseModel:Using weights: {'STU': [1, 1, 1]}


AL access metrics for STU complete!
Skipping tl_2010_us_state10.zip, it's already downloaded.
Skipping tl_2010_us_state10.zip, it's already extracted.
Skipping tl_2010_35_tract10.zip, it's already downloaded.
Skipping tl_2010_35_tract10.zip, it's already extracted.
Skipping tl_2010_32_tract10.zip, it's already downloaded.
Skipping tl_2010_32_tract10.zip, it's already extracted.
Skipping tl_2010_49_tract10.zip, it's already downloaded.
Skipping tl_2010_49_tract10.zip, it's already extracted.
Skipping tl_2010_08_tract10.zip, it's already downloaded.
Skipping tl_2010_08_tract10.zip, it's already extracted.
Skipping tl_2010_06_tract10.zip, it's already downloaded.
Skipping tl_2010_06_tract10.zip, it's already extracted.
Skipping tl_2010_04_tract10.zip, it's already downloaded.
Skipping tl_2010_04_tract10.zip, it's already extracted.
AZ destinations saved to ./inputs/tract/AZ/AZ-STU-moud-dests.csv


AZ transformed matrix saved to ./inputs/tract/AZ/AZ-transformed-matrix.csv


...

WI transformed matrix saved to ./inputs/tract/WI/WI-transformed-matrix.csv


INFO:spatial_access.BaseModel:Using weights: {'STU': [1, 1, 1]}


WI access metrics for STU complete!
Skipping tl_2010_us_state10.zip, it's already downloaded.
Skipping tl_2010_us_state10.zip, it's already extracted.
Skipping tl_2010_56_tract10.zip, it's already downloaded.
Skipping tl_2010_56_tract10.zip, it's already extracted.
Skipping tl_2010_31_tract10.zip, it's already downloaded.
Skipping tl_2010_31_tract10.zip, it's already extracted.
Skipping tl_2010_49_tract10.zip, it's already downloaded.
Skipping tl_2010_49_tract10.zip, it's already extracted.
Skipping tl_2010_08_tract10.zip, it's already downloaded.
Skipping tl_2010_08_tract10.zip, it's already extracted.
Skipping tl_2010_30_tract10.zip, it's already downloaded.
Skipping tl_2010_30_tract10.zip, it's already extracted.
Skipping tl_2010_16_tract10.zip, it's already downloaded.
Skipping tl_2010_16_tract10.zip, it's already extracted.
Skipping tl_2010_46_tract10.zip, it's already downloaded.
Skipping tl_2010_46_tract10.zip, it's already extracted.
WY destinations saved to ./inputs/tract/WY/W

INFO:spatial_access.BaseModel:Using weights: {'STU': [1, 1, 1]}


WY transformed matrix saved to ./inputs/tract/WY/WY-transformed-matrix.csv
WY access metrics for STU complete!


In [122]:
access_metrics = dd.read_csv(f'./outputs/tract/*/*-{resource[:3]}-access-metrics.csv')

In [123]:
access_metrics.to_csv(f'./outputs/tract/contigUS-{resource[:3]}-access-metrics.csv', single_file = True, index=False)

['/home/vidal/Projects/compumatrix/outputs/tract/contigUS-nal-access-metrics.csv']

In [124]:
resource = 'nal'

In [125]:
us = f'/home/vidal/Projects/compumatrix/outputs/tract/contigUS-{resource[:3]}-access-metrics.csv'

In [126]:
us_dd = dd.read_csv(us)
us_dd.columns
us_dd['tract'] = us_dd['tract'].astype(str)

In [127]:
us_dd.head()

,tract,time_to_nearest_naltrexone/vivitrol,count_in_range_naltrexone/vivitrol,naltrexone/vivitrol_score
0,1087232000,49,0,0.0
1,1103005701,33,0,0.0
2,1081040901,53,0,0.0
3,1081040902,53,0,0.0
4,1023957000,74,0,0.0


In [128]:
us_dd[us_dd['tract'].str.startswith("60")].compute()

,tract,time_to_nearest_naltrexone/vivitrol,count_in_range_naltrexone/vivitrol,naltrexone/vivitrol_score
3393,6037504001,4,97,2.100000
3394,6037504002,4,102,2.200000
3395,6037405701,6,64,2.566667
3396,6037405702,7,64,2.600000
3397,6037209102,0,179,2.800000
...,...,...,...,...
11445,6085509108,15,80,2.100000
11446,6085509109,14,80,2.266667
11447,6037700600,5,178,2.533333
11448,6037602301,3,178,2.466667


In [129]:
us_dd[us_dd['tract'].str.startswith('48')].compute()

,tract,time_to_nearest_naltrexone/vivitrol,count_in_range_naltrexone/vivitrol,naltrexone/vivitrol_score
61112,48201531400,12,43,2.300000
61113,48201433100,10,43,2.066667
61114,48213950500,61,0,0.233333
61115,48041001000,99,0,1.333333
61116,48185180200,83,0,0.000000
...,...,...,...,...
66372,48349970400,80,0,0.000000
66373,48121020401,6,6,2.400000
66374,48121020402,0,6,2.700000
66375,48121020403,4,6,2.800000


In [130]:
us_dd['tract'] = us_dd['tract'].str.zfill(11)

In [131]:
us_dd[us_dd['tract'].str.startswith("06")].compute()

,tract,time_to_nearest_naltrexone/vivitrol,count_in_range_naltrexone/vivitrol,naltrexone/vivitrol_score
3393,06037504001,4,97,2.100000
3394,06037504002,4,102,2.200000
3395,06037405701,6,64,2.566667
3396,06037405702,7,64,2.600000
3397,06037209102,0,179,2.800000
...,...,...,...,...
11445,06085509108,15,80,2.100000
11446,06085509109,14,80,2.266667
11447,06037700600,5,178,2.533333
11448,06037602301,3,178,2.466667


In [132]:
us_dd.to_csv(f'/home/vidal/Projects/compumatrix/outputs/tract/contigUS-{resource[:3]}-access-metrics.csv', single_file=True)

['/home/vidal/Projects/compumatrix/outputs/tract/contigUS-nal-access-metrics.csv']